In [1]:
## For debbugging
%load_ext autoreload

In [2]:
# import os
import sys
from pathlib import Path
module_path = Path('.').resolve().parents[1]
if module_path not in sys.path:
    sys.path.append(str(module_path))


In [3]:
## for debugging:
# %aimport mlp.keyword
# %aimport mlp.tools
%aimport mlp.kex
%aimport app.test_app
# %aimport mlp.tree
%autoreload 1
from mlp.kex import *


In [4]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, tqdm
%matplotlib inline

In [5]:
%gui qt
import PyQt5.QtWidgets as qw
from app.test_app import MyWindow
from PyQt5 import QtCore



In [6]:
import dask.dataframe as dd
import dask
import string
from fuzzywuzzy import process as zz

In [9]:
data_dir = module_path/'data'/'auto_data'

df = dd.read_csv(data_dir/'auto_data.csv',
                 dtype={'JPNUM': 'object'})


nlp_select = NLPSelect(columns = ['DESCRIPTION', 'DESCRIPTION_1'])

raw_text = nlp_select.transform(df)

### Build Vocab 

In [10]:
tex = TokenExtractor()
toks = tex.fit_transform(raw_text)
print(tex.vocab_)

['mod' 'op' 'completed' ... 'supplier' 'req' 'pipefitters']


In [28]:
vocab_fname = data_dir/'vocab.csv'
# vocab_fname = data_dir/'mine_vocab_app.csv'

vocab = tex.annotation_assistant(filename=vocab_fname)

In [29]:
if __name__ == '__main__':

    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = app = qw.QApplication(sys.argv)

    window = MyWindow(vocab_filename=vocab_fname)
    window.show()

    app.exec_()

In [30]:
vocab = tex.annotation_assistant(init=vocab_fname)

### Extract Keywords

In [31]:

# vocab = pd.read_csv(data_dir/'app_vocab_mike.csv', index_col=0)
# how many instances of each keyword class are there?
print('named entities: ')
print('I\tItem\nP\tProblem\nS\tSolution\nR\tRedundant')
print('U\tUnknown\nX\tStop Word')
print('total tokens: ', vocab.NE.notna().sum())
print('total tags: ', vocab.groupby("NE").nunique().alias.sum())
vocab.groupby("NE").nunique()

named entities: 
I	Item
P	Problem
S	Solution
R	Redundant
U	Unknown
X	Stop Word
total tokens:  35
total tags:  10


,NE,alias,notes,score
NE,,,,
I,1,7,3,23
P,1,1,0,6
U,1,1,0,1
X,1,1,0,5


In [33]:
# # vocab.loc['replaced',:]
# tag_df = pd.DataFrame(index=range(len(raw_text)))

tag_set = pd.DataFrame(index=range(len(raw_text)))
tag_df = pd.DataFrame(index=range(len(raw_text)))
for clf, queries in tqdm_notebook(vocab.fillna({'NE':'NA', 
                                                'alias':vocab.index.to_series()}).groupby('NE').alias.unique().iteritems(), 
                                  desc='Category Loop', total=vocab.NE.nunique()):
    clf_df = pd.DataFrame(index=range(len(raw_text)))
    for query in tqdm_notebook(queries, desc='token loop', leave=True):
    #     query = 'hydraulic'
    #     vocab[vocab.alias==query].index.tolist()
        query_idx = [tex._model.vocabulary_[i] for i in vocab[vocab.alias==query].index.tolist()]
        match = ((toks[:,query_idx]).toarray() > 0).any(axis=1).astype(int)
        clf_df[query] = match
        tag_df[query] = match

    
    tag_set[clf] = clf_df.apply(lambda x: ', '.join(x[x==1].index.astype(str)), axis=1)
# vocab.groupby('NE').alias.unique()